## Установка нужных версий библиотек

In [1]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt

### 1. Получение данных load_breat_cancer

Будем работать с набором данных для задачи регрессии `load_diabetes`, который можно получить из стандартных датасетов в `sklearn'e`.

После `load_diabetes()` возвращается словарь с данными (`data`), целевой переменной (`target`), названиями характеристик в данных (`feature_names`) и описанием данных (`DESCR`).

In [2]:
from sklearn.datasets import load_diabetes

data = load_diabetes()
data

{'data': array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
          0.01990749, -0.01764613],
        [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
         -0.06833155, -0.09220405],
        [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
          0.00286131, -0.02593034],
        ...,
        [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
         -0.04688253,  0.01549073],
        [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
          0.04452873, -0.02593034],
        [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
         -0.00422151,  0.00306441]]),
 'target': array([151.,  75., 141., 206., 135.,  97., 138.,  63., 110., 310., 101.,
         69., 179., 185., 118., 171., 166., 144.,  97., 168.,  68.,  49.,
         68., 245., 184., 202., 137.,  85., 131., 283., 129.,  59., 341.,
         87.,  65., 102., 265., 276., 252.,  90., 100.,  55.,  61.,  92.,
        259.,  53., 190., 142.,  75., 142., 155., 225.,  59

In [3]:
X = data.data
features = data.feature_names
y = data.target

Из признаков (характеристик данных) и целевой переменной сформируем датафрейм, в качестве названий колонок возьмем названия признаков.

In [4]:
df = pd.DataFrame(X, columns=features)
df['target'] = y

df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


Разобьем выборку на две: обучающую и тестовую.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df[features],
    df['target'],
    test_size=0.2,
    shuffle=True,
    random_state=3
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((353, 10), (353,), (89, 10), (89,))

### 1.1. Обучение дерева решений

1. Инициализируйте дерево решений для задачи регрессии
2. Обучите его на обучающей выборке

In [6]:
tree = DecisionTreeRegressor(random_state=3)
tree.fit(X_train, y_train)

DecisionTreeRegressor(random_state=3)

In [40]:
tree.get_depth()

8

In [41]:
tree.get_n_leaves()

100

### 1.2. Получение метрик качества
Узнаем, насколько дерево решений обучилось хорошо, для этого
1. Сделайте предсказания моделью для обучающей выборки
2. Сделайте предсказания моделью для тестовой выборки
3. Посчитайте метрику качества средне-квадратичная ошибка
4. Посчитайте метрику качества коэффициент детерминации

In [7]:
# Предсказания моделью для обучающей выборки
train_predict = tree.predict(X_train)
train_predict[:5]

array([ 89., 160., 109., 198.,  72.])

In [8]:
# Предсказания моделью для тестовой выборки
test_predict = tree.predict(X_test)
test_predict[:5]

array([ 63., 141., 138.,  42., 196.])

In [9]:
# Считаем метрику качества mse
train_mse = mean_squared_error(y_train, train_predict)
test_mse = mean_squared_error(y_test, test_predict)
mse_0 = pd.DataFrame({'train': train_mse, 'test': test_mse}, index=['mse'])
mse_0

,train,test
mse,0.0,5670.820225


In [10]:
# Считаем коэффициент детерминации
train_r2 = r2_score(y_train, train_predict)
test_r2 = r2_score(y_test, test_predict)
r2_0 = pd.DataFrame({'train': train_r2, 'test': test_r2}, index=['r2'])
r2_0

,train,test
r2,1.0,-0.047301


Сделайте вывод, насколько хорошо обучилась модель

**Вывод**: налицо переобучение модели, т.к. значения метрик модели на обучающей выборке идеальны, а на тестовой неудовлетворительные.

### 1.3. Изменение метрики

Попробуйте поизменять известные параметры для того, чтобы метрика стала лучше.

#### 1.3.1 Параметр max_depth

In [11]:
df_i = mse_0
for i in range(2, 20):
    tree = DecisionTreeRegressor(random_state=3, max_depth=i)
    tree.fit(X_train, y_train)
    train_predict = tree.predict(X_train)
    test_predict = tree.predict(X_test)
    train_mse = mean_squared_error(y_train, train_predict)
    test_mse = mean_squared_error(y_test, test_predict)
    df_i = pd.concat([df_i, pd.DataFrame({'train': train_mse, 'test': test_mse}, index=[f"max_depth = {i}"])])
df_i

,train,test
mse,0.000000,5670.820225
max_depth = 2,3395.236119,3423.442294
max_depth = 3,2948.860709,3391.964822
max_depth = 4,2488.519738,3476.471247
max_depth = 5,1978.971974,3859.725919
max_depth = 6,1500.521678,4722.448125
max_depth = 7,1083.550354,4689.702413
max_depth = 8,760.960703,5290.732710
max_depth = 9,430.209868,5455.983075
max_depth = 10,239.902428,5546.520928


In [12]:
# изменим значения параметра max_depth
df_i = r2_0
for i in range(2, 20):
    tree = DecisionTreeRegressor(random_state=3, max_depth=i)
    tree.fit(X_train, y_train)
    train_predict = tree.predict(X_train)
    test_predict = tree.predict(X_test)
    train_r2 = r2_score(y_train, train_predict)
    test_r2 = r2_score(y_test, test_predict)
    df_i = pd.concat([df_i, pd.DataFrame({'train': train_r2, 'test': test_r2}, index=[f"max_depth = {i}"])])
df_i

,train,test
r2,1.000000,-0.047301
max_depth = 2,0.438768,0.367750
max_depth = 3,0.512554,0.373563
max_depth = 4,0.588648,0.357957
max_depth = 5,0.672876,0.287176
max_depth = 6,0.751964,0.127846
max_depth = 7,0.820889,0.133894
max_depth = 8,0.874213,0.022894
max_depth = 9,0.928886,-0.007625
max_depth = 10,0.960344,-0.024345


**Вывод**: наиболее приемлемый результат модель показала при значинии параметра глубины дерева равного трем. На этом уровне удалось улучшить качество предсказаний модели на тестовой выборке, по сравнению с базовыми настройками. Далее значения метрик качества в тестовой подбыворке прогрессивно ухудшались, одновренменно с их улучшением в обучающей выборке, что указывает на переобучение модели.

#### Параметр min_samples_leaf

In [23]:
df_i = mse_0
for i in range(1, 20):
    tree = DecisionTreeRegressor(random_state=3, min_samples_leaf=i)
    tree.fit(X_train, y_train)
    train_predict = tree.predict(X_train)
    test_predict = tree.predict(X_test)
    train_mse = mean_squared_error(y_train, train_predict)
    test_mse = mean_squared_error(y_test, test_predict)
    df_i = pd.concat([df_i, pd.DataFrame({'train': train_mse, 'test': test_mse}, index=[f"min_samples_leaf = {i}"])])

df_i

,train,test
mse,0.000000,5670.820225
min_samples_leaf = 1,0.000000,16700.112229
min_samples_leaf = 2,473.974446,15997.328209
min_samples_leaf = 3,1148.387713,15430.179592
min_samples_leaf = 4,1900.629717,15039.896455
min_samples_leaf = 5,2630.151160,14634.111660
min_samples_leaf = 6,3274.746040,14270.335239
min_samples_leaf = 7,3853.279752,14194.600402
min_samples_leaf = 8,4383.206440,14040.598652
min_samples_leaf = 9,4845.107818,13933.656992


#### Параметр max_leaf_nodes

In [34]:
df_i = mse_0
for i in range(80, 101):
    tree = DecisionTreeRegressor(random_state=3, max_leaf_nodes=i)
    tree.fit(X_train, y_train)
    train_predict = tree.predict(X_train)
    test_predict = tree.predict(X_test)
    train_mse = mean_squared_error(y_train, train_predict)
    test_mse = mean_squared_error(y_test, test_predict)
    df_i = pd.concat([df_i, pd.DataFrame({'train': train_mse, 'test': test_mse}, index=[f"max_leaf_nodes = {i}"])])

df_i

,train,test
mse,0.000000,5670.820225
max_leaf_nodes = 80,22834.556169,23792.683082
max_leaf_nodes = 81,22783.474787,23794.656157
max_leaf_nodes = 82,22732.566467,23760.033005
max_leaf_nodes = 83,22681.918825,23713.139707
max_leaf_nodes = 84,22631.501676,23677.204579
max_leaf_nodes = 85,22581.317996,23621.879049
max_leaf_nodes = 86,22531.886098,23540.503165
max_leaf_nodes = 87,22483.093685,23490.900375
max_leaf_nodes = 88,22434.676504,23428.350358


### 2. Получение данных make_regression

Для второго примера возьмем самодельный набор данных для задачи регрессии `make_regression`, который можно получить из стандартных датасетов в `sklearn'e`.

Сгенерируем себе 100к объектов, которые описываются 20 признаками, из них 12 будут дейтсвительно полезными.

In [14]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples=100_000, n_features=20, n_informative=12, random_state=10)

In [15]:
X.shape, y.shape

((100000, 20), (100000,))

Разобьем выборку на две: обучающую и тестовую.

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    shuffle=True,
    random_state=3
)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((80000, 20), (80000,), (20000, 20), (20000,))

### 2.1. Обучение дерева решений

1. Инициализируйте дерево решений для задачи регрессии
2. Обучите его на обучающей выборке

In [35]:
tree_2 = DecisionTreeRegressor(random_state=3)
tree_2.fit(X_train, y_train)

DecisionTreeRegressor(random_state=3)

In [36]:
train_predicts = tree_2.predict(X_train)
test_predicts = tree_2.predict(X_test)

### 2.2. Получение метрик качества
Узнаем, насколько дерево решений обучилось хорошо, для этого
1. Сделайте предсказания моделью для обучающей выборки
2. Сделайте предсказания моделью для тестовой выборки
3. Посчитайте метрику качества средне-квадратичная ошибка
4. Посчитайте метрику качества коэффициент детерминации

In [39]:
mse_train = mean_squared_error(y_train, train_predicts)
mse_test = mean_squared_error(y_test, test_predicts)
pd.DataFrame({'train': mse_train, 'test': mse_test}, index=['mse'])

,train,test
mse,0.0,16700.112229


Сделайте вывод, насколько хорошо обучилась модель

In [19]:
# Ваш вывод здесь

### 2.3. Изменение метрики

Попробуйте поизменять известные параметры для того, чтобы метрика стала лучше.

In [20]:
# Ваш код здесь

In [21]:
# Ваш вывод здесь

Сегодня на практическом занятии:
1. Обучили дерево решений без критериев останова
2. Изменяли знакомые три критерия останова, чтобы снизить переобучение
3. Обучили две модели